<a href="https://colab.research.google.com/github/SreyaMaxwel/Snack_Spy/blob/main/Snack_Spy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
# Install necessary packages
!pip install -q streamlit rembg opencv-python-headless onnxruntime pyngrok


In [37]:
%%writefile app.py
import cv2
import numpy as np
from rembg import remove
from PIL import Image
import streamlit as st
import io

def preprocess_image(file):
    input_image = Image.open(file).convert("RGB")
    output_image = remove(input_image)
    img_array = np.array(output_image)
    gray = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
    _, binary = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    return binary

def calculate_food_area(binary_mask):
    return np.sum(binary_mask == 255)

def main():
    st.set_page_config(page_title="Food Quantity Comparator", layout="wide")
    st.title("🍲 Food Quantity Comparator")
    st.markdown("Upload two food images to **compare their quantity visually** using computer vision.")

    col1, col2 = st.columns(2)
    with col1:
        img1 = st.file_uploader("Upload First Image", type=["jpg", "jpeg", "png"], key="1")
    with col2:
        img2 = st.file_uploader("Upload Second Image", type=["jpg", "jpeg", "png"], key="2")

    if img1 and img2:
        st.subheader("🔍 Preprocessing Images...")
        binary1 = preprocess_image(img1)
        binary2 = preprocess_image(img2)

        area1 = calculate_food_area(binary1)
        area2 = calculate_food_area(binary2)

        similarity = min(area1, area2) / max(area1, area2)

        st.write("---")
        st.subheader("🖼 Original Images")
        col3, col4 = st.columns(2)
        with col3:
            st.image(img1, caption="Image 1", use_column_width=True)
        with col4:
            st.image(img2, caption="Image 2", use_column_width=True)

        st.write("---")
        st.subheader("📐 Analysis")
        st.write(f"📏 **Image 1 Area:** {area1} pixels")
        st.write(f"📏 **Image 2 Area:** {area2} pixels")
        st.write(f"📊 **Similarity:** {similarity * 100:.2f}%")

        st.write("---")
        if similarity > 0.9:
            st.success("✅ Quantities are **very similar or the same**!")
        elif similarity > 0.7:
            st.warning("⚠️ Quantities are **somewhat similar**, but not exact.")
        else:
            st.error("❌ Quantities are **different**.")

if __name__ == "__main__":
    main()


Overwriting app.py


In [38]:
import threading
import time
from pyngrok import ngrok

ngrok.kill()

def run():
    !streamlit run app.py &> /content/log.txt

thread = threading.Thread(target=run)
thread.start()
time.sleep(5)

public_url = ngrok.connect(8501)
print(f"🚀 Your app is live at: {public_url}")


🚀 Your app is live at: NgrokTunnel: "https://708785a2ee9a.ngrok-free.app" -> "http://localhost:8501"
